# E-commerce Customer Profiling and Data Consolidation Project

# Problem Statement

An e-commerce startup, after a successful first month, is seeking to better understand its customer base and purchasing patterns. They aim to answer fundamental questions like "Who are our customers?" "What do they buy?" and "What drives their purchasing behavior?" However, they face challenges in accurately identifying and counting customers due to disparate data sources, including:

1. A customer database recording online account sign-ups,
2. A CRM system tracking phone and non-online customer interactions, and
3. Raw transaction data that includes guest purchases with no formal customer record.

These sources of data may have overlapping entries or duplicate records, often due to customers engaging with multiple systems (e.g., making a purchase as a guest, then creating an account later). Furthermore, customer information may vary across entries due to potential discrepancies like typos or alternative identifiers. The startup seeks the assistance of an analyst to address these complexities and provide an accurate count and profile of their customers.


## Project Goals

1. **Data Consolidation:**  
   Integrate customer information across all available sources to create a unified, comprehensive dataset representing the complete customer base.

2. **Data Deduplication:**  
   Identify and resolve duplicate records, maintaining data lineage to track original sources and identifiers for each customer where possible.

3. **Customer Identification:**  
   Develop a data model that uniquely identifies each customer, regardless of which data source(s) they appear in, and can be easily queried for counts and profiles.

4. **Data Validation:**  
   Ensure data integrity by confirming assumptions, identifying inconsistencies, and validating data completeness to maximize reliability.

5. **Solution Documentation:**  
   Create a flexible and traceable solution, with a well-defined schema that supports future analysis of customer demographics and purchasing behavior.

This project will ultimately enable the startup to answer critical questions about their customer demographics and purchasing trends, creating a foundation for data-driven decision-making.

To actually perform the data modeling, we should take the following course of action:

1. First we explore all 3 datasets. We should see if the values in all the columns make sense.

2. Then we can trim down our data before we merge our different datasets, so that we have unique customers

3. Then we merge these trimmed datasets and remove duplicates carefully.

4. When performing deduplication, we need to go one step beyond and identify more than just exact match duplicates. We can use Fuzzy Matching here.

5. Finally, we clean and present our data model.

## Exploring our Purchases Dataset

In [1]:
import pandas as pd
import numpy as np

sales = pd.read_csv("../data/purchases.csv")
print(sales.shape)

(71519, 11)


In [2]:
sales.head()


,event_time,product_id,category_id,category_code,brand,price,session_id,customer_id,guest_first_name,guest_surname,guest_postcode
0,2022-10-01 02:26:08+00:00,32701106,2055156924466332447,NaN,shimano,95.21,64c68405-7002-4ce0-9604-a4c2e1f7384b,NaN,MICHAEL,MASON,RG497ZQ
1,2022-10-01 02:28:32+00:00,9400066,2053013566067311601,NaN,jaguar,164.20,3b7d6741-3c82-4c75-8015-6f54b52612e0,7466.0,NaN,NaN,NaN
2,2022-10-01 02:31:01+00:00,1004238,2053013555631882655,electronics.smartphone,apple,1206.40,38c6d3f7-6c32-4fed-bca6-ef98e1746386,NaN,COLE,WILKINSON,SW75TQ
3,2022-10-01 02:33:31+00:00,11300059,2053013555531219353,electronics.telephone,texet,17.48,3398c966-7846-4186-89be-323daad735b9,NaN,MOHAMMED,RICHARDS,RG150RE
4,2022-10-01 02:40:18+00:00,17300751,2053013553853497655,NaN,versace,77.22,11e3a573-01b9-4794-b513-e7d8a4fcac83,31266.0,NaN,NaN,NaN


In [3]:
sales.isnull().sum()

event_time              0
product_id              0
category_id             0
category_code       16739
brand                5707
price                   0
session_id              0
customer_id         18448
guest_first_name    53071
guest_surname       53071
guest_postcode      53071
dtype: int64

Looks like the 18448 missing customer IDs (which are guest checkouts) and the 53071 missing guest values (which are registered customers) make up all of our data. We should verify there's no overlap, e.g. a row with both customer ID and guest details missing.

Let's create a column to track guest checkouts:

In [4]:
sales["is_guest"] = sales["customer_id"].isnull()

Let's check cases where a guest checkout also had a customer id filled

In [5]:
sales[sales["is_guest"] & sales["customer_id"].notnull()]

,event_time,product_id,category_id,category_code,brand,price,session_id,customer_id,guest_first_name,guest_surname,guest_postcode,is_guest


no such case. Let's also check cases where we neither have a guest checkout nor a customer id filled.

In [6]:
sales[(sales["is_guest"] == False) & sales["customer_id"].isnull()]

,event_time,product_id,category_id,category_code,brand,price,session_id,customer_id,guest_first_name,guest_surname,guest_postcode,is_guest


This tells us that either all rows are a guest checkout or a purchase made by a registered customer. Let's now check what percentage of records are guest checkouts.

In [7]:
sales['is_guest'].value_counts(normalize=True)

False    0.742055
True     0.257945
Name: is_guest, dtype: float64

Now we know around 25% of our records are guest checkouts. We need to keep in mind that each row in our dataset represents a purchased item and not a customer record. Hence, this won't be the actual proportion of guest checkouts. Now, to calculate the actual proportion:

In [8]:
guest_columns = ["guest_first_name", "guest_surname", "guest_postcode"]
unique_guests = sales[guest_columns].drop_duplicates()
print(len(unique_guests))

unique_customers = sales["customer_id"].unique()
cust_total = len(unique_customers) + len(unique_guests) - 1 # Subtracting 1 because Null is also counted once
print(len(unique_customers))
print(cust_total)

8301
24962
33262


## Exporting Customer Data from Purchases

We need to create a schema where we can store our customer records. If we look at our datasets, we can see that customer and CRM datasets have names, postcodes and age (which is not present in our guest customers, but we should not discard this column just because its not present in guests). We can also keep track of where our records came from by adding an indicator column for each data source. We could also just have one column called sources with values like 'purchases', 'CRM' or 'customer database' but it would not help us if we have duplicate customers in different sources.

Our data model schema will contain these columns:
customer_id, first_name, surname, postcode, age, is_guest, in_purchase_data, in_crm_data, in_customer_data

To extract customer data from purchases, we should first extract the guests and non-guests separately and then combine both.

In [9]:
guest_columns = ['guest_first_name', 'guest_surname', 'guest_postcode', 'is_guest']
guests = sales.loc[sales['is_guest'], guest_columns]
guests = guests.drop_duplicates()
guests.head()

,guest_first_name,guest_surname,guest_postcode,is_guest
0,MICHAEL,MASON,RG497ZQ,True
2,COLE,WILKINSON,SW75TQ,True
3,MOHAMMED,RICHARDS,RG150RE,True
7,KIAN,MILLS,SW332TF,True
13,RUBY,OWEN,PO377YS,True


In [10]:
non_guests = (pd.DataFrame(sales.loc[sales["customer_id"].notnull(), "customer_id"]
                          .unique()
                          .astype(int),
                          columns = ['customer_id']
                          ))
non_guests.head()

,customer_id
0,7466
1,31266
2,534142828
3,1035
4,6985


In [11]:
sales_customers = pd.concat([non_guests, guests], axis=0, ignore_index=True)

In [12]:
new_col_names = ["customer_id", "first_name", "surname","postcode", "is_guest"]
sales_customers = sales_customers.set_axis(new_col_names, axis=1)

In [13]:
sales_customers["is_guest"] = sales_customers["is_guest"].fillna(False)

In [14]:
sales_customers["in_purchase_data"] = True

In [15]:
for col in ["first_name", "surname"]:
    sales_customers[col] = sales_customers[col].str.lower().str.strip()


sales_customers["postcode"] = sales_customers["postcode"].str.strip()

We have successfully extracted customers data from purchases.

## Exploring CRM Data

In [16]:
crm = pd.read_csv("../data/crm_export.csv")
print(crm.shape)
crm.head()

(7825, 5)


,customer_id,first_name,surname,postcode,age
0,29223,Holly,Rogers,LS475RT,12
1,27826,Daniel,Owen,M902XX,5
2,7432,Eleanor,Russell,HR904ZA,34
3,2569,Paige,Roberts,DE732EP,61
4,9195,Matilda,Young,LS670FU,78


In [17]:
crm.isnull().sum()


customer_id    0
first_name     0
surname        0
postcode       0
age            0
dtype: int64

In [18]:
crm.isnull().sum()

customer_id    0
first_name     0
surname        0
postcode       0
age            0
dtype: int64

Sanitizing string columns

In [19]:
for col in ["first_name", "surname"]:
    crm[col] = crm[col].str.lower().str.strip()

crm["postcode"] = crm["postcode"].str.strip()

In [20]:
# Verifying that customer ID is unique
crm.groupby("customer_id").size().loc[lambda x: x > 1]

Series([], dtype: int64)

In [21]:
# Checking for duplicate information
print(len(crm))
print(len(crm.drop(columns="customer_id").drop_duplicates()))


7825
7419


In [22]:
print("{} rows".format(len(crm)))
print("{} unique combinations of customer information".format(len(crm.drop(columns="customer_id").drop_duplicates())))

7825 rows
7419 unique combinations of customer information


Now let's join CRM data to purchases where purchases have a customer ID (not guests)

In [23]:
print(len(sales_customers))
sales_customers.isnull().sum()

33261


customer_id          8300
first_name          24961
surname             24961
postcode            24961
is_guest                0
in_purchase_data        0
dtype: int64

In [24]:
sales_and_crm_customers = sales_customers.merge(crm, on="customer_id", how="left", suffixes=("_sales", "_crm"))
print(len(sales_and_crm_customers))
sales_and_crm_customers.isnull().sum()

33261


customer_id          8300
first_name_sales    24961
surname_sales       24961
postcode_sales      24961
is_guest                0
in_purchase_data        0
first_name_crm      26147
surname_crm         26147
postcode_crm        26147
age                 26147
dtype: int64

In [25]:
# now we have duplicate customer detail columns, so merge them into the old ones
merged_customers_filter = (
    (sales_and_crm_customers["customer_id"].notnull()) # only for actual customers
    # only if they have at least first or surname filled in
    # meaning we've found a match in our CRM data
    & ((sales_and_crm_customers["first_name_crm"].notnull())
       | (sales_and_crm_customers["surname_crm"].notnull()))
)

sales_and_crm_customers.loc[merged_customers_filter, "in_crm_data"] = True
sales_and_crm_customers.loc[~merged_customers_filter, "in_crm_data"] = False

sales_and_crm_customers["in_crm_data"].value_counts()

False    26147
True      7114
Name: in_crm_data, dtype: int64

Now merge data into single versions of customer information

In [26]:
sales_and_crm_customers.loc[merged_customers_filter, ["first_name_sales", "surname_sales", "postcode_sales"]] = (
    sales_and_crm_customers.loc[merged_customers_filter, ["first_name_crm", "surname_crm", "postcode_crm"]]
    .values
)

# drop duplicate columns and rename
sales_and_crm_customers = (
    sales_and_crm_customers
    .drop(columns=["first_name_crm", "surname_crm", "postcode_crm"])
    .rename(columns={
        "first_name_sales": "first_name",
        "surname_sales": "surname",
        "postcode_sales": "postcode"
    })
)

print(sales_and_crm_customers.isnull().sum())
sales_and_crm_customers.head()

customer_id          8300
first_name          17847
surname             17847
postcode            17847
is_guest                0
in_purchase_data        0
age                 26147
in_crm_data             0
dtype: int64


,customer_id,first_name,surname,postcode,is_guest,in_purchase_data,age,in_crm_data
0,7466.0,NaN,NaN,NaN,False,True,NaN,False
1,31266.0,harley,palmer,HR250EJ,False,True,33.0,True
2,534142828.0,NaN,NaN,NaN,False,True,NaN,False
3,1035.0,NaN,NaN,NaN,False,True,NaN,False
4,6985.0,NaN,NaN,NaN,False,True,NaN,False


Now, we will add CRM customers not in our purchase history

In [27]:
crm_ids_to_add = list(set(crm["customer_id"].unique()) - set(sales_and_crm_customers["customer_id"].unique()))
print(len(crm_ids_to_add))

711


In [28]:
print(len(sales_and_crm_customers))

sales_and_crm_customers = (
    pd.concat([sales_and_crm_customers, crm[crm["customer_id"].isin(crm_ids_to_add)]],
              axis=0,
             ignore_index=True)
)

print(len(sales_and_crm_customers))

33261
33972


In [29]:
sales_and_crm_customers.isnull().sum()

customer_id          8300
first_name          17847
surname             17847
postcode            17847
is_guest              711
in_purchase_data      711
age                 26147
in_crm_data           711
dtype: int64

Updating missing record sources accordingly

In [30]:
sales_and_crm_customers["is_guest"] = sales_and_crm_customers["is_guest"].fillna(False)
sales_and_crm_customers["in_purchase_data"] = sales_and_crm_customers["in_purchase_data"].fillna(False)
sales_and_crm_customers["in_crm_data"] = sales_and_crm_customers["in_crm_data"].fillna(True)

sales_and_crm_customers.isnull().sum()

customer_id          8300
first_name          17847
surname             17847
postcode            17847
is_guest                0
in_purchase_data        0
age                 26147
in_crm_data             0
dtype: int64

In [31]:
sales_and_crm_customers["in_crm_data"].value_counts()

False    26147
True      7825
Name: in_crm_data, dtype: int64

In [32]:
print(len(sales_and_crm_customers))
sales_and_crm_customers.head()

33972


,customer_id,first_name,surname,postcode,is_guest,in_purchase_data,age,in_crm_data
0,7466.0,NaN,NaN,NaN,False,True,NaN,False
1,31266.0,harley,palmer,HR250EJ,False,True,33.0,True
2,534142828.0,NaN,NaN,NaN,False,True,NaN,False
3,1035.0,NaN,NaN,NaN,False,True,NaN,False
4,6985.0,NaN,NaN,NaN,False,True,NaN,False


There is now one row per customer from BOTH the CRM data and purchase history

## Customers

In [33]:
customers = pd.read_csv("../data/customer_database.csv")
print(customers.shape)
customers.head()

(23476, 5)


,customer_id,first_name,surname,postcode,age
0,1641,Rhys,Richards,DE456EZ,45
1,24796,Maisie,Young,SW433XX,16
2,14358,Nathan,King,NW49TU,58
3,15306,Jack,Moore,NW908RR,26
4,24971,Alexander,Roberts,SW500HW,85


In [34]:
customers.isnull().sum()

customer_id    0
first_name     0
surname        0
postcode       0
age            0
dtype: int64

In [35]:
for col in ["first_name", "surname"]:
    customers[col] = customers[col].str.lower().str.strip()

customers["postcode"] = customers["postcode"].str.strip()

In [36]:
customers.groupby("customer_id").size().loc[lambda x: x>1]

Series([], dtype: int64)

In [37]:
print("{} rows".format(len(customers)))
unique_customers = customers.drop(columns="customer_id").drop_duplicates()
print("{} unique combinations of customers".format(len(unique_customers)))

23476 rows
19889 unique combinations of customers


Joining this data to our merged customer data

In [38]:
print(len(sales_and_crm_customers))

all_customers = sales_and_crm_customers.merge(customers, on="customer_id", how="left", suffixes=("_sales", "_customers"))
print(len(all_customers))
all_customers.head()

33972
33972


,customer_id,first_name_sales,surname_sales,postcode_sales,is_guest,in_purchase_data,age_sales,in_crm_data,first_name_customers,surname_customers,postcode_customers,age_customers
0,7466.0,NaN,NaN,NaN,False,True,NaN,False,eve,richards,HR90PT,45.0
1,31266.0,harley,palmer,HR250EJ,False,True,33.0,True,NaN,NaN,NaN,NaN
2,534142828.0,NaN,NaN,NaN,False,True,NaN,False,NaN,NaN,NaN,NaN
3,1035.0,NaN,NaN,NaN,False,True,NaN,False,luca,gibson,DE256NH,30.0
4,6985.0,NaN,NaN,NaN,False,True,NaN,False,mia,rogers,HR662RP,43.0


In [39]:
# now we have duplicate customer detail columns, so merge them into the old ones
merged_customers_filter = (
    (all_customers["customer_id"].notnull()) # only for actual customers
    # only if they have at least first or surname filled in
    & ((all_customers["first_name_customers"].notnull())
       | (all_customers["surname_customers"].notnull()))
)

all_customers.loc[merged_customers_filter, "in_customer_data"] = True
all_customers.loc[~merged_customers_filter, "in_customer_data"] = False

all_customers["in_customer_data"].value_counts()

True     22053
False    11919
Name: in_customer_data, dtype: int64

In [40]:
all_customers.isnull().sum()

customer_id              8300
first_name_sales        17847
surname_sales           17847
postcode_sales          17847
is_guest                    0
in_purchase_data            0
age_sales               26147
in_crm_data                 0
first_name_customers    11919
surname_customers       11919
postcode_customers      11919
age_customers           11919
in_customer_data            0
dtype: int64

If customers are newly added, fill in customer info:

In [41]:
update_filter = (
    (all_customers["in_customer_data"])
    & (all_customers["first_name_sales"].isnull())
    & (all_customers["surname_sales"].isnull())
)

print(len(all_customers))

all_customers.loc[update_filter, ["first_name_sales", "surname_sales", "postcode_sales", "age_sales"]] = (
    all_customers.loc[update_filter, ["first_name_customers", "surname_customers", "postcode_customers", "age_customers"]].values
)

all_customers = (
    all_customers
    .drop(columns=["first_name_customers", "surname_customers", "age_customers", "postcode_customers"])
    .rename(columns={
        "first_name_sales": "first_name",
        "surname_sales": "surname",
        "age_sales": "age",
        "postcode_sales": "postcode"
    })
)

print(len(all_customers))
all_customers.isnull().sum()

33972
33972


customer_id         8300
first_name          1248
surname             1248
postcode            1248
is_guest               0
in_purchase_data       0
age                 9548
in_crm_data            0
in_customer_data       0
dtype: int64

Add customers from customer DB but not in main data

In [42]:
customer_ids_to_add = list(set(customers["customer_id"].unique()) - set(all_customers["customer_id"].unique()))
print(len(customer_ids_to_add))

1423


In [43]:
print(len(all_customers))

all_customers = (
    pd.concat([all_customers, customers[customers["customer_id"].isin(customer_ids_to_add)]],
              axis=0,
              ignore_index=True)
)

print(len(all_customers))

33972
35395


In [44]:
all_customers.isnull().sum()

customer_id         8300
first_name          1248
surname             1248
postcode            1248
is_guest            1423
in_purchase_data    1423
age                 9548
in_crm_data         1423
in_customer_data    1423
dtype: int64

At this point if we have missing data for whether someone was a guest, it means they weren't (since we can only mark guests using the purchase data and we've already done that)

In [45]:
all_customers["is_guest"] = all_customers["is_guest"].fillna(False)
all_customers["in_purchase_data"] = all_customers["in_purchase_data"].fillna(False)
all_customers["in_crm_data"] = all_customers["in_crm_data"].fillna(False)
all_customers["in_customer_data"] = all_customers["in_customer_data"].fillna(True)

all_customers.isnull().sum()

customer_id         8300
first_name          1248
surname             1248
postcode            1248
is_guest               0
in_purchase_data       0
age                 9548
in_crm_data            0
in_customer_data       0
dtype: int64

In [46]:
all_customers.head()

,customer_id,first_name,surname,postcode,is_guest,in_purchase_data,age,in_crm_data,in_customer_data
0,7466.0,eve,richards,HR90PT,False,True,45.0,False,True
1,31266.0,harley,palmer,HR250EJ,False,True,33.0,True,False
2,534142828.0,NaN,NaN,NaN,False,True,NaN,False,False
3,1035.0,luca,gibson,DE256NH,False,True,30.0,False,True
4,6985.0,mia,rogers,HR662RP,False,True,43.0,False,True


In [47]:
len(all_customers)

35395

## Deduplicate

Let's first remove exact duplicates.

In [49]:
print(len(all_customers))
all_customers = all_customers.drop_duplicates()
print(len(all_customers))

35395
35395


Let's count up all the following cases:

- those purchased with a customer ID (whether or not that ID is in another database)
- those purchased as guests
- those in the CRM database who haven't made a purchase
- those in the customer database who haven't made a purchase


In [50]:
identified_customers = (
    all_customers[(all_customers["customer_id"].notnull())
                  & (all_customers["in_purchase_data"])
                  & ((all_customers["in_crm_data"])
                     | (all_customers["in_customer_data"]))]
)

guests = all_customers[all_customers["is_guest"]]

customer_ids_not_found = (
    all_customers[(all_customers["customer_id"].notnull())
                  & (all_customers["first_name"].isnull())
                  & (all_customers["surname"].isnull())]
)

customer_data_only = (
    all_customers[((all_customers["in_crm_data"])
                   | (all_customers["in_customer_data"])
                  )
                  & (all_customers["in_purchase_data"] == False)]
)

print(len(all_customers), len(identified_customers))
print(len(guests), len(customer_ids_not_found), len(customer_data_only))

35395 23713
8300 1248 2134


In [51]:
print(f"Size of customer database (not yet deduplicated): {len(all_customers)}")
print(f"Identified customers (with an ID, present in a lookup DB): {len(identified_customers)}")
print(f"Guest checkouts: {len(guests)}")
print(f"Unidentified customer IDs in purchases: {len(customer_ids_not_found)}")
print(f"Customers with no purchases present in CRM or customer data: {len(customer_data_only)}")

Size of customer database (not yet deduplicated): 35395
Identified customers (with an ID, present in a lookup DB): 23713
Guest checkouts: 8300
Unidentified customer IDs in purchases: 1248
Customers with no purchases present in CRM or customer data: 2134


In [52]:
assert len(all_customers) == len(identified_customers) + len(guests) + len(customer_ids_not_found) + len(customer_data_only)
